In [16]:
import pandas as pd 
import numpy as np
import os
from sklearn.model_selection import train_test_split
from string import printable 
from keras.utils import pad_sequences
import tensorflow as tf
import json
from tensorflow.keras.models import model_from_json
from pathlib import Path
import tensorflow as tf
from keras.models import Model
from keras import regularizers, Sequential
from keras.layers import Dense, Dropout, Activation, Lambda, Flatten, Input, ELU, LSTM, Embedding, BatchNormalization, Conv1D, concatenate, MaxPooling1D
from keras.preprocessing import sequence
from keras.optimizers import Adam
from keras.utils import np_utils
from keras import backend as K
from sklearn.preprocessing import LabelEncoder

In [3]:
def read_data():
  df = pd.read_csv("malicious_phish.csv")
  url_int_tokens = [
      [printable.index(x) + 1 for x in url if x in printable] for url in df.iloc[:, 0]
  ]

  max_len = 75
  X = pad_sequences(url_int_tokens, maxlen=max_len)
  le1 = LabelEncoder()

  df['type'] = le1.fit_transform(df['type'])
  target = np.array(df['type'])
  x_train, x_test, target_train, target_test = train_test_split(X, target, test_size=0.25, random_state=42)

  return x_train, x_test, target_train, target_test


In [6]:
x_train, x_test, target_train, target_test = read_data()

array([3, 0, 0, ..., 1, 0, 0])

In [5]:
max_len = 75
emb_dim = 32
max_vocab_len = 101
lstm_output_size = 32
W_reg = regularizers.l2(1e-4)
epochs_num = 10
batch_size = 32

In [ ]:
def save_model(model, fileModelJSON, fileWeights):
  if Path(fileModelJSON).is_file():
        os.remove(fileModelJSON)
  json_string = model.to_json()
  with open(fileModelJSON, 'w') as f:
      json.dump(json_string, f)

  if Path(fileWeights).is_file():
      os.remove(fileWeights)
  model.save_weights(fileWeights)


def load_model(fileModelJSON, fileWeights):
    with open(fileModelJSON, 'r') as f:
        model_json = json.load(f)
        model = model_from_json(model_json)

    model.load_weights(fileWeights)
    return model

# CNN

In [15]:
model1 = Sequential()
model1.add(Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len, embeddings_regularizer=W_reg))
model1.add(Conv1D(kernel_size=2, filters=256, padding='same', activation='elu'))
model1.add(Conv1D(kernel_size=3, filters=256, padding='same', activation='elu'))
model1.add(Conv1D(kernel_size=4, filters=256, padding='same', activation='elu'))
model1.add(Conv1D(kernel_size=5, filters=256, padding='same', activation='elu'))
model1.add(Dense(1024))
model1.add(ELU())
model1.add(BatchNormalization())
model1.add(Dense(1024))
model1.add(ELU())
model1.add(BatchNormalization())
model1.add(Dense(1, activation='sigmoid'))

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
y_train = np.asarray(target_train).astype('float32').reshape((-1,1))
y_test = np.asarray(target_test).astype('float32').reshape((-1, 1))
model1.fit(x_train, y_train, epochs=epochs_num, batch_size=batch_size)


Epoch 1/10
15263/15263 [==============================] - 294s 19ms/step - loss: -0.4483 - accuracy: 0.4130
Epoch 2/10
15263/15263 [==============================] - 261s 17ms/step - loss: -0.8669 - accuracy: 0.4794
Epoch 3/10
15263/15263 [==============================] - 258s 17ms/step - loss: -1.0284 - accuracy: 0.4931
Epoch 4/10
15263/15263 [==============================] - 261s 17ms/step - loss: -1.1228 - accuracy: 0.4973
Epoch 5/10
15263/15263 [==============================] - 262s 17ms/step - loss: -1.2106 - accuracy: 0.5066
Epoch 6/10
15263/15263 [==============================] - 257s 17ms/step - loss: -1.2736 - accuracy: 0.5120
Epoch 7/10
15263/15263 [==============================] - 248s 16ms/step - loss: -1.3224 - accuracy: 0.5168
Epoch 8/10
15263/15263 [==============================] - 248s 16ms/step - loss: -1.3636 - accuracy: 0.5201
Epoch 9/10
15263/15263 [==============================] - 249s 16ms/step - loss: -1.4059 - accuracy: 0.5235
Epoch 10/10
15263/15263 [===

In [17]:
loss, accuracy = model1.evaluate(x_test, y_test, verbose=0)
print("Final cross validation accuracy =", accuracy)

Final cross validation accuracy = 0.4807218313217163


# Convolutional LSTM

In [18]:
model2 = Sequential()
model2.add(Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len, embeddings_regularizer=W_reg))
model2.add(Conv1D(kernel_size=5, filters=256, padding='same', activation='elu'))
model2.add(MaxPooling1D(pool_size=4))
model2.add(Dropout(0.5))
model2.add(LSTM(lstm_output_size))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(x_train, target_train, epochs=epochs_num, batch_size=batch_size)

Epoch 1/10
15263/15263 [==============================] - 89s 6ms/step - loss: -47.5746 - accuracy: 0.6221
Epoch 2/10
15263/15263 [==============================] - 85s 6ms/step - loss: -148.1568 - accuracy: 0.6199
Epoch 3/10
15263/15263 [==============================] - 85s 6ms/step - loss: -251.0025 - accuracy: 0.6222
Epoch 4/10
15263/15263 [==============================] - 86s 6ms/step - loss: -354.5500 - accuracy: 0.6232
Epoch 5/10
15263/15263 [==============================] - 86s 6ms/step - loss: -459.5931 - accuracy: 0.6241
Epoch 6/10
15263/15263 [==============================] - 93s 6ms/step - loss: -559.8657 - accuracy: 0.6254
Epoch 7/10
15263/15263 [==============================] - 102s 7ms/step - loss: -667.5125 - accuracy: 0.6270
Epoch 8/10
15263/15263 [==============================] - 98s 6ms/step - loss: -770.2989 - accuracy: 0.6283
Epoch 9/10
15263/15263 [==============================] - 97s 6ms/step - loss: -873.0135 - accuracy: 0.6269
Epoch 10/10
15263/15263 [===

In [19]:
loss, accuracy = model2.evaluate(x_test, y_test, verbose=0)
print("Final cross validation accuracy =", accuracy)

Final cross validation accuracy = 0.6768940687179565


# Simple LSTM

In [ ]:
model3 = Sequential()
model3.add(Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len, embeddings_regularizer=W_reg))
model3.add(LSTM(lstm_output_size))
model3.add(Dense(1, activation='sigmoid'))

print(model3.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 75, 32)            3232      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 11,585
Trainable params: 11,585
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
x_valid, y_valid = x_train[:batch_size], target_train[:batch_size]
x_train2, y_train2 = x_train[batch_size:], target_train[batch_size:]
model3.fit(x_train2, y_train2, validation_data=(x_valid, y_valid), batch_size=batch_size, epochs=epochs_num)

Epoch 1/10
15262/15262 [==============================] - 95s 6ms/step - loss: -15.7385 - accuracy: 0.5644 - val_loss: -43.6021 - val_accuracy: 0.6562
Epoch 2/10
15262/15262 [==============================] - 93s 6ms/step - loss: -97.6877 - accuracy: 0.6403 - val_loss: -132.8496 - val_accuracy: 0.5938
Epoch 3/10
15262/15262 [==============================] - 94s 6ms/step - loss: -205.5326 - accuracy: 0.6586 - val_loss: -193.5416 - val_accuracy: 0.6562
Epoch 4/10
15262/15262 [==============================] - 93s 6ms/step - loss: -309.2408 - accuracy: 0.6837 - val_loss: -298.3667 - val_accuracy: 0.6875
Epoch 5/10
15262/15262 [==============================] - 92s 6ms/step - loss: -411.1554 - accuracy: 0.6835 - val_loss: -382.7057 - val_accuracy: 0.7812
Epoch 6/10
15262/15262 [==============================] - 93s 6ms/step - loss: -501.7339 - accuracy: 0.6663 - val_loss: -467.4328 - val_accuracy: 0.7188
Epoch 7/10
15262/15262 [==============================] - 96s 6ms/step - loss: -622.1

In [ ]:
loss, accuracy = model1.evaluate(x_test, target_test, verbose=0)
print("Final cross validation accuracy =", accuracy)

Final cross validation accuracy = 0.7038599848747253
